In [ ]:
pip install --upgrade langchain langchain-google-genai chromadb


In [ ]:
%pip install --upgrade --quiet  langchain-community

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
API_KEY = "GOOGLE_API_KEY"

In [ ]:
# 1. Document text (embedded in code)
doc_text = """
Elon Musk is a technology entrepreneur and engineer known for founding SpaceX and Tesla.
He was born on June 28, 1971, in Pretoria, South Africa.
His major achievements include advancing space exploration and electric vehicles.
Musk is also involved with Neuralink and The Boring Company.
This document provides a brief overview of Musk's background and accomplishments.
"""

document = Document(page_content=doc_text, metadata={"source": "in-memory-doc"})

In [ ]:
# 2. Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs_split = text_splitter.split_documents([document])

In [ ]:
# 3. Use SentenceTransformers embeddings via HuggingFaceEmbeddings wrapper
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-7-3586352694.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models o

In [ ]:
# 4. Create FAISS vector store
vectorstore = FAISS.from_documents(docs_split, embeddings)


In [ ]:
# 5. Initialize Gemini 2.5 Flash LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    max_tokens=None,
    api_key=GOOGLE_API_KEY,
)

In [ ]:
# 6. Setup RetrievalQA chain
retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type="stuff",
)

In [ ]:
# 7. Query the system
query = "Who is Elon Musk and what are his major achievements?"
result = qa_chain.invoke({"query": query})

print("Answer:", result['result'])
print("\nSource Documents:")
for doc in result['source_documents']:
    print(f"- {doc.page_content}")

Answer: Elon Musk is a technology entrepreneur and engineer known for founding SpaceX and Tesla. His major achievements include advancing space exploration and electric vehicles.

Source Documents:
- Elon Musk is a technology entrepreneur and engineer known for founding SpaceX and Tesla.
He was born on June 28, 1971, in Pretoria, South Africa.
His major achievements include advancing space exploration and electric vehicles.
Musk is also involved with Neuralink and The Boring Company.
This document provides a brief overview of Musk's background and accomplishments.
